<a href="https://colab.research.google.com/github/cuzmyk/data_analysis/blob/main/DA_3pr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Практическая работа №3. Применение методов глубокого обучения в среде Apache Spark**:

In [ ]:
%%capture
!pip install elephas
!pip install findspark
!pip install numpy==1.26.0

# Перезапуск среды выполнения
import os
os.kill(os.getpid(), 9)

# После установки библиотек появится сообщение "Сеанс прекращен по неизвестной причине."
# Так и должно быть. Запускайте код в следующих далее ячейках.

In [1]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.optimizers.legacy import SGD
from tensorflow.keras.utils import to_categorical

from elephas.spark_model import SparkModel
from elephas.utils.rdd_utils import to_simple_rdd

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
import findspark

findspark.init()

In [2]:
conf = SparkConf().setMaster("local").setAppName("Data Analysis")
sc = SparkContext(conf = conf)
spark = SparkSession(sc)

In [3]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import optimizers
from tensorflow.keras.optimizers.legacy import SGD

from elephas.ml_model import ElephasEstimator
from elephas.ml.adapter import to_data_frame

from pyspark import SparkContext, SparkConf
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.ml import Pipeline

from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType, FloatType

import numpy as np

### **Задание 1. Обучите модель классификации**

Ссылка на датасеты: https://www.kaggle.com/datasets?search=classification&tags=13302-Classification

In [4]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, StandardScaler, VectorAssembler

In [5]:
df = spark.read.csv('/content/drug200.csv', header=True, inferSchema=True)
df.printSchema()
df.show(5)

root
 |-- Age: integer (nullable = true)
 |-- Sex: string (nullable = true)
 |-- BP: string (nullable = true)
 |-- Cholesterol: string (nullable = true)
 |-- Na_to_K: double (nullable = true)
 |-- Drug: string (nullable = true)

+---+---+------+-----------+-------+-----+
|Age|Sex|    BP|Cholesterol|Na_to_K| Drug|
+---+---+------+-----------+-------+-----+
| 23|  F|  HIGH|       HIGH| 25.355|DrugY|
| 47|  M|   LOW|       HIGH| 13.093|drugC|
| 47|  M|   LOW|       HIGH| 10.114|drugC|
| 28|  F|NORMAL|       HIGH|  7.798|drugX|
| 61|  F|   LOW|       HIGH| 18.043|DrugY|
+---+---+------+-----------+-------+-----+
only showing top 5 rows



In [6]:
from pyspark.ml.feature import StringIndexer
indexer_sex = StringIndexer(inputCol='Sex', outputCol='Sex_index')
indexer_bp = StringIndexer(inputCol='BP', outputCol='BP_index')
indexer_ch = StringIndexer(inputCol='Cholesterol', outputCol='Cholesterol_index')
indexer_drug = StringIndexer(inputCol='Drug', outputCol='label')

df = indexer_sex.fit(df).transform(df)
df = indexer_bp.fit(df).transform(df)
df = indexer_ch.fit(df).transform(df)
df = indexer_drug.fit(df).transform(df)

In [7]:
encoder = OneHotEncoder(
    inputCols=["Sex_index", "BP_index", "Cholesterol_index"],
    outputCols=["SexVec", "BPVec", "CholVec"]
)
df = encoder.fit(df).transform(df)

In [8]:
assembler = VectorAssembler(
    inputCols=['Age', 'Sex_index', 'BP_index', 'Cholesterol_index', 'Na_to_K'],
    outputCol='features'
)

In [9]:
output = assembler.transform(df)
output.select("features").show()

+--------------------+
|            features|
+--------------------+
|[23.0,1.0,0.0,0.0...|
|[47.0,0.0,1.0,0.0...|
|[47.0,0.0,1.0,0.0...|
|[28.0,1.0,2.0,0.0...|
|[61.0,1.0,1.0,0.0...|
|[22.0,1.0,2.0,0.0...|
|[49.0,1.0,2.0,0.0...|
|[41.0,0.0,1.0,0.0...|
|[60.0,0.0,2.0,0.0...|
|[43.0,0.0,1.0,1.0...|
|[47.0,1.0,1.0,0.0...|
|[34.0,1.0,0.0,1.0...|
|[43.0,0.0,1.0,0.0...|
|[74.0,1.0,1.0,0.0...|
|[50.0,1.0,2.0,0.0...|
|[16.0,1.0,0.0,1.0...|
|[69.0,0.0,1.0,1.0...|
|(5,[0,4],[43.0,13...|
|[23.0,0.0,1.0,0.0...|
|[32.0,1.0,0.0,1.0...|
+--------------------+
only showing top 20 rows



In [10]:
output.show()

+---+---+------+-----------+-------+-----+---------+--------+-----------------+-----+-------------+-------------+-------------+--------------------+
|Age|Sex|    BP|Cholesterol|Na_to_K| Drug|Sex_index|BP_index|Cholesterol_index|label|       SexVec|        BPVec|      CholVec|            features|
+---+---+------+-----------+-------+-----+---------+--------+-----------------+-----+-------------+-------------+-------------+--------------------+
| 23|  F|  HIGH|       HIGH| 25.355|DrugY|      1.0|     0.0|              0.0|  0.0|    (1,[],[])|(2,[0],[1.0])|(1,[0],[1.0])|[23.0,1.0,0.0,0.0...|
| 47|  M|   LOW|       HIGH| 13.093|drugC|      0.0|     1.0|              0.0|  4.0|(1,[0],[1.0])|(2,[1],[1.0])|(1,[0],[1.0])|[47.0,0.0,1.0,0.0...|
| 47|  M|   LOW|       HIGH| 10.114|drugC|      0.0|     1.0|              0.0|  4.0|(1,[0],[1.0])|(2,[1],[1.0])|(1,[0],[1.0])|[47.0,0.0,1.0,0.0...|
| 28|  F|NORMAL|       HIGH|  7.798|drugX|      1.0|     2.0|              0.0|  1.0|    (1,[],[])|    (2,

In [11]:
final_data = output.select("features",'label')

In [12]:
# Разделение на обучающую и тестовую выборку
train_data, test_data = final_data.randomSplit([0.7,0.3])

In [13]:
# Создание модели Keras
model = Sequential()
model.add(Dense(64, input_shape=(train_data.select('features').first()[0].size,)))
model.add(Activation('relu'))
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dense(5, activation='softmax'))  # 5 классов (типов лекарств)

In [14]:
# Компиляция модели
from tensorflow.keras.optimizers import Adam

model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [15]:
# Настройка оптимизатора
adam_optimizer = Adam(learning_rate=0.0001)
adam_conf = optimizers.serialize(adam_optimizer)

In [16]:
def to_data_frame(spark, x_data, y_data):
    return spark.createDataFrame(
        [(Vectors.dense(x), int(y)) for x, y in zip(x_data, y_data)],
        ['features', 'label']
    )

In [17]:
# Преобразуем данные для обучения и тестирования
from pyspark.ml.linalg import Vectors

train_features = [Vectors.dense(row[0].toArray()) for row in train_data.select('features').collect()]
train_labels = [row[0] for row in train_data.select('label').collect()]
df_train = spark.createDataFrame(zip(train_features, train_labels), ["features", "label"])

test_features = [Vectors.dense(row[0].toArray()) for row in test_data.select('features').collect()]
test_labels = [row[0] for row in test_data.select('label').collect()]
df_test = spark.createDataFrame(zip(test_features, test_labels), ["features", "label"])

In [18]:
# Создание и настройка ElephasEstimator
estimator = ElephasEstimator()
estimator.set_keras_model_config(model.to_json())
estimator.set_optimizer_config({"class_name": "Adam", "config": {"learning_rate": 0.0001}})
estimator.set_mode("asynchronous")
estimator.set_loss("sparse_categorical_crossentropy")
estimator.set_metrics(['accuracy'])
estimator.set_epochs(30)
estimator.set_batch_size(2)
estimator.set_validation_split(0.1)
estimator.set_categorical_labels(False)

ElephasEstimator_3111c9c4639d

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                384       
                                                                 
 activation (Activation)     (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                                 
 activation_1 (Activation)   (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 5)                 325       
                                                                 
Total params: 4869 (19.02 KB)
Trainable params: 4869 (19.02 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [20]:
from pyspark.ml import Pipeline
import numpy as np
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType
from pyspark.mllib.evaluation import MulticlassMetrics

In [21]:
# Обучаем модель через ElephasEstimator
pipeline = Pipeline(stages=[estimator])

# Обучение на тренировочном наборе
fitted_pipeline = pipeline.fit(df_train)

>>> Fit model
 * Serving Flask app 'elephas.parameter.server'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://172.28.0.12:4000
INFO:werkzeug:Press CTRL+C to quit


>>> Initialize workers
>>> Distribute load


INFO:werkzeug:172.28.0.12 - - [03/Jun/2025 17:44:21] "GET /parameters HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.12 - - [03/Jun/2025 17:44:22] "POST /update HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.12 - - [03/Jun/2025 17:44:22] "GET /parameters HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.12 - - [03/Jun/2025 17:44:22] "POST /update HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.12 - - [03/Jun/2025 17:44:22] "GET /parameters HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.12 - - [03/Jun/2025 17:44:22] "POST /update HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.12 - - [03/Jun/2025 17:44:22] "GET /parameters HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.12 - - [03/Jun/2025 17:44:22] "POST /update HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.12 - - [03/Jun/2025 17:44:22] "GET /parameters HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.12 - - [03/Jun/2025 17:44:23] "POST /update HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.12 - - [03/Jun/2025 17:44:23] "GET /parameters HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.12 - - [03/Jun/2025 17:44:23] "POST /update HT

>>> Async training complete.


In [22]:
# Применение на тестовом наборе
predictions = fitted_pipeline.transform(df_test)

In [23]:
# UDF-функция для получения класса из вероятностей
argmax_udf = udf(lambda x: int(np.argmax(x)), IntegerType())

# Применим UDF
predictions = predictions.withColumn("prediction", argmax_udf(predictions["prediction"]))

In [24]:
# Отобразим первые 10 строк предсказаний
predictions.select("label", "prediction").show(10, truncate=False)

+-----+----------+
|label|prediction|
+-----+----------+
|2.0  |1         |
|0.0  |0         |
|2.0  |0         |
|2.0  |1         |
|0.0  |0         |
|0.0  |0         |
|3.0  |1         |
|1.0  |0         |
|0.0  |0         |
|2.0  |0         |
+-----+----------+
only showing top 10 rows



In [25]:
# Оценка качества
prediction_and_labels = predictions.select("prediction", "label").rdd.map(lambda row: (float(row.prediction), float(row.label)))

metrics = MulticlassMetrics(prediction_and_labels)

# Матрица ошибок
print("Матрица ошибок:")
print(metrics.confusionMatrix().toArray())

# Точность модели
accuracy = metrics.accuracy
print(f"\nТочность модели: {accuracy:.4f}")

# Precision и Recall по каждому классу
labels = prediction_and_labels.map(lambda x: x[1]).distinct().collect()
for label in labels:
    print(f"Класс {int(label)}:")
    print(f"  Precision: {metrics.precision(label):.4f}")
    print(f"  Recall: {metrics.recall(label):.4f}")
    print(f"  F1-score: {metrics.fMeasure(label):.4f}")

/usr/local/lib/python3.11/dist-packages/pyspark/sql/context.py:158: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


Матрица ошибок:
[[29.  4.  0.  0.  0.]
 [ 4. 13.  0.  0.  0.]
 [ 5.  3.  0.  0.  0.]
 [ 0.  6.  0.  0.  0.]
 [ 0.  5.  0.  0.  0.]]

Точность модели: 0.6087
Класс 2:
  Precision: 0.0000
  Recall: 0.0000
  F1-score: 0.0000
Класс 1:
  Precision: 0.4194
  Recall: 0.7647
  F1-score: 0.5417
Класс 4:
  Precision: 0.0000
  Recall: 0.0000
  F1-score: 0.0000
Класс 0:
  Precision: 0.7632
  Recall: 0.8788
  F1-score: 0.8169
Класс 3:
  Precision: 0.0000
  Recall: 0.0000
  F1-score: 0.0000


### **Задание 2. Обучите модель регрессии**

Ссылка на датасеты: https://www.kaggle.com/datasets?search=Regression&page=2

In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.linalg import Vectors
from elephas.ml_model import ElephasEstimator
from elephas.ml.adapter import to_data_frame
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import SGD
import tensorflow.keras.optimizers as optimizers

In [ ]:
df_2 = spark.read.csv('/content/kc_house_data.csv', header=True, inferSchema=True)
df_2.printSchema()
df_2.show(5)

root
 |-- id: long (nullable = true)
 |-- date: string (nullable = true)
 |-- price: double (nullable = true)
 |-- bedrooms: integer (nullable = true)
 |-- bathrooms: double (nullable = true)
 |-- sqft_living: integer (nullable = true)
 |-- sqft_lot: integer (nullable = true)
 |-- floors: double (nullable = true)
 |-- waterfront: integer (nullable = true)
 |-- view: integer (nullable = true)
 |-- condition: integer (nullable = true)
 |-- grade: integer (nullable = true)
 |-- sqft_above: integer (nullable = true)
 |-- sqft_basement: integer (nullable = true)
 |-- yr_built: integer (nullable = true)
 |-- yr_renovated: integer (nullable = true)
 |-- zipcode: integer (nullable = true)
 |-- lat: double (nullable = true)
 |-- long: double (nullable = true)
 |-- sqft_living15: integer (nullable = true)
 |-- sqft_lot15: integer (nullable = true)

+----------+---------------+--------+--------+---------+-----------+--------+------+----------+----+---------+-----+----------+-------------+--------

In [ ]:
# Преобразуем данные в нужный формат
feature_columns = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
                   'waterfront', 'view', 'condition', 'grade', 'sqft_above',
                   'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long']


In [ ]:
# Создаем VectorAssembler
assembler = VectorAssembler(inputCols=feature_columns, outputCol='features')

In [ ]:
df_transformed = assembler.transform(df_2)

In [ ]:
# Разделяем данные на обучающую и тестовую выборки
train_df, test_df = df_transformed.randomSplit([0.8, 0.2], seed=1234)

In [ ]:
# Преобразуем RDD в два отдельных списка: features и labels
train_features = train_df.select("features").rdd.map(lambda row: row["features"].toArray()).collect()
train_labels = train_df.select("price").rdd.map(lambda row: row["price"]).collect()

test_features = test_df.select("features").rdd.map(lambda row: row["features"].toArray()).collect()
test_labels = test_df.select("price").rdd.map(lambda row: row["price"]).collect()

# Преобразуем в формат для Elephas
train_data = to_data_frame(spark.sparkContext, train_features, train_labels)
test_data = to_data_frame(spark.sparkContext, test_features, test_labels)


In [ ]:
# Построение модели Keras
model = Sequential()
model.add(Dense(64, input_shape=(len(feature_columns),)))
model.add(Activation('relu'))
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dense(1))  # Линейная регрессия

In [ ]:
batch_size = 32
epochs = 50

In [ ]:
# Конфигурация оптимизатора
from keras.optimizers.legacy import SGD
sgd = SGD(learning_rate=0.000001)

sgd_conf = {
    "class_name": "SGD",
    "config": sgd.get_config()
}

In [ ]:
estimator = ElephasEstimator()
estimator.set_keras_model_config(model.to_json())
estimator.set_optimizer_config(sgd_conf)
estimator.set_mode("synchronous")
estimator.set_loss("mae")
estimator.set_metrics(['mse'])
estimator.set_epochs(epochs)
estimator.set_batch_size(batch_size)
estimator.set_validation_split(0.1)
estimator.set_categorical_labels(False)

ElephasEstimator_b3ab83f56b0a

In [ ]:
# Обучение модели через Pipeline
pipeline = Pipeline(stages=[estimator])
fitted_pipeline = pipeline.fit(train_data)

>>> Fit model
>>> Synchronous training complete.


In [ ]:
# Предсказания
prediction = fitted_pipeline.transform(test_data)
pnl = prediction.select("label", "prediction")
pnl.show(10)

+--------+------------+
|   label|  prediction|
+--------+------------+
|300000.0|435159.71875|
|235000.0|371819.59375|
|617000.0| 400429.9375|
|843000.0| 475154.9375|
|837700.0|474464.53125|
|715000.0| 461884.6875|
|795000.0|  470488.875|
|319500.0| 416803.4375|
|570000.0|  397235.375|
|550120.0|  427566.375|
+--------+------------+
only showing top 10 rows



In [ ]:
from pyspark.mllib.evaluation import RegressionMetrics

prediction_and_label = pnl.rdd.map(lambda row: (row.label, row.prediction))
metrics = RegressionMetrics(prediction_and_label)

print("R^2:", metrics.r2)
print("MAE:", metrics.meanAbsoluteError)
print("RMSE:", metrics.rootMeanSquaredError)

R^2: -34.116089512249985
MAE: 205432.4489170146
RMSE: 352278.4227038646


### **Задание 3. Обучите модель текстовой классификации**

Ссылка на датасеты: https://www.kaggle.com/datasets?search=text+classification

In [43]:
from pyspark.ml.feature import Tokenizer, HashingTF, IDF
from pyspark.ml import Pipeline
from pyspark.sql.functions import col, when
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers.legacy import Adam
from elephas.spark_model import SparkModel
from pyspark.mllib.evaluation import MulticlassMetrics
import numpy as np

In [44]:
df = spark.read.csv('/content/ecommerceDataset.csv', header=True, inferSchema=True)
df.printSchema()
df.show(5)

root
 |-- Household: string (nullable = true)
 |-- Paper Plane Design Framed Wall Hanging Motivational Office Decor Art Prints (8.7 X 8.7 inch) - Set of 4 Painting made up in synthetic frame with uv textured print which gives multi effects and attracts towards it. This is an special series of paintings which makes your wall very beautiful and gives a royal touch. This painting is ready to hang, you would be proud to possess this unique painting that is a niche apart. We use only the most modern and efficient printing technology on our prints, with only the and inks and precision epson, roland and hp printers. This innovative hd printing technique results in durable and spectacular looking prints of the highest that last a lifetime. We print solely with top-notch 100% inks, to achieve brilliant and true colours. Due to their high level of uv resistance, our prints retain their beautiful colours for many years. Add colour and style to your living space with this digitally printed paintin

In [45]:
# Обработка null и пустых значений
df = df.withColumn("Household", when(col("Household").isNull(), "").otherwise(col("Household")))
df = df.filter(col("Household") != "")
print(f"Количество строк после очистки: {df.count()}")

Количество строк после очистки: 50455


In [46]:
# Создание сбалансированной выборки (по 250 строк на класс)
classes = ['Household', 'Books', 'Clothing & Accessories', 'Electronics']
balanced_df = None
for cls in classes:
    cls_df = df.filter(col("Household") == cls).limit(250)
    if balanced_df is None:
        balanced_df = cls_df
    else:
        balanced_df = balanced_df.union(cls_df)

print(f"Количество строк в сбалансированной выборке: {balanced_df.count()}")

Количество строк в сбалансированной выборке: 1000


In [47]:
# Кодирование меток через StringIndexer
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer(inputCol="Household", outputCol="indexedLabel")
indexer_model = indexer.fit(balanced_df)
train_df = indexer_model.transform(balanced_df)

In [48]:
# Проверка данных
train_df.select("Household", "indexedLabel").show(2, truncate=50)
train_df.groupBy("indexedLabel").count().show()

+---------+------------+
|Household|indexedLabel|
+---------+------------+
|Household|         3.0|
|Household|         3.0|
+---------+------------+
only showing top 2 rows

+------------+-----+
|indexedLabel|count|
+------------+-----+
|         3.0|  250|
|         0.0|  250|
|         1.0|  250|
|         2.0|  250|
+------------+-----+



In [49]:
# Предобработка текста
tokenizer = Tokenizer(inputCol="Household", outputCol="words")
hashingTF = HashingTF(inputCol="words", outputCol="raw_features", numFeatures=2000)
idf = IDF(inputCol="raw_features", outputCol="features")

In [50]:
# Пайплайн
pipeline = Pipeline(stages=[tokenizer, hashingTF, idf])
pipeline_model = pipeline.fit(train_df)
train_data = pipeline_model.transform(train_df)

In [51]:
# Разделение данных
train_data, test_data = train_data.randomSplit([0.7, 0.3], seed=42)
train_data.cache()
test_data.cache()

print(f"Train count: {train_data.count()}")
print(f"Test count: {test_data.count()}")

Train count: 705
Test count: 295


In [52]:
# Создание модели Keras
num_classes = 4  # Household, Books, Clothing & Accessories, Electronics
model = Sequential()
model.add(Dense(128, input_shape=(2000,)))
model.add(Activation('relu'))
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [53]:
# Компиляция модели
model.compile(optimizer=Adam(lr=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Сохранение и загрузка модели
model.save('temp_model_ecommerce.h5')
from tensorflow.keras.models import load_model
model = load_model('temp_model_ecommerce.h5')

In [54]:
# Подготовка RDD
train_rdd = train_data.rdd.map(lambda row: (row['features'].toArray(), row['indexedLabel']))

In [55]:
from tensorflow.keras.callbacks import EarlyStopping

spark_model = SparkModel(
    model,
    frequency='batch',  # Более частая синхронизация (чем 'epoch') может улучшить сходимость
    mode='asynchronous',  # Асинхронный режим часто работает быстрее для больших кластеров
    port=4001,
    num_workers=4,  # Оптимальное количество воркеров для вашего кластера
    elastic_learning_rate=0.001,  # Адаптация learning rate между воркерами
    dynamic_loss_scale=True  # Автоматическое масштабирование loss для стабильности
)

history = spark_model.fit(
    train_rdd,
    epochs=50,  # Больше эпох для сложных моделей
    batch_size=128,  # Увеличенный batch для распределённого обучения
    validation_split=0.1,  # Меньше validation данных для большего обучающего набора
    verbose=1,
    shuffle=True,  # Перемешивание данных между эпохами
    learning_rate=0.001,  # Стандартный learning rate для Adam
    metrics=['accuracy'],  # Контроль метрик
    callbacks=[EarlyStopping(monitor='val_loss', patience=3)]  # Ранняя остановка
)

>>> Fit model
 * Serving Flask app 'elephas.parameter.server'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://172.28.0.12:4001
INFO:werkzeug:Press CTRL+C to quit


>>> Initialize workers
>>> Distribute load


INFO:werkzeug:172.28.0.12 - - [03/Jun/2025 17:40:38] "GET /parameters HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.12 - - [03/Jun/2025 17:40:38] "POST /update HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.12 - - [03/Jun/2025 17:40:38] "GET /parameters HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.12 - - [03/Jun/2025 17:40:38] "POST /update HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.12 - - [03/Jun/2025 17:40:38] "GET /parameters HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.12 - - [03/Jun/2025 17:40:38] "POST /update HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.12 - - [03/Jun/2025 17:40:38] "GET /parameters HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.12 - - [03/Jun/2025 17:40:38] "POST /update HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.12 - - [03/Jun/2025 17:40:38] "GET /parameters HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.12 - - [03/Jun/2025 17:40:38] "POST /update HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.12 - - [03/Jun/2025 17:40:38] "GET /parameters HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.12 - - [03/Jun/2025 17:40:38] "POST /update HT

>>> Async training complete.


In [56]:
# Предсказания
test_np_x = np.array(test_data.rdd.map(lambda row: row['features'].toArray()).collect())
test_np_y = np.array(test_data.rdd.map(lambda row: row['indexedLabel']).collect())
predictions = spark_model.predict(test_np_x)
predictions = np.argmax(predictions, axis=1)

In [57]:
# Метрики
print(f"test_np_x shape: {test_np_x.shape}")
print(f"test_np_y shape: {test_np_y.shape}")
print(f"predictions shape: {predictions.shape}")

prediction_and_label = spark.sparkContext.parallelize([(float(y), float(p)) for y, p in zip(test_np_y, predictions)])
metrics = MulticlassMetrics(prediction_and_label)
print(f"Accuracy: {metrics.accuracy}")
print(f"Precision: {metrics.weightedPrecision}")
print(f"Recall: {metrics.weightedRecall}")
print(f"F1 Score: {metrics.weightedFMeasure()}")

print(f"Sample preds: {predictions[:10]}")
print(f"Sample actual: {test_np_y[:10]}")

test_np_x shape: (295, 2000)
test_np_y shape: (295,)
predictions shape: (295,)
Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1 Score: 1.0
Sample preds: [3 3 3 3 3 3 3 3 3 3]
Sample actual: [3. 3. 3. 3. 3. 3. 3. 3. 3. 3.]


In [58]:
# Завершение сессии
spark.stop()